In [3]:
#巴尔干边界
import torch

## 1. 定义
RNN的本质是拥有记忆的能力，并且会根据记忆的内容进行推断，因此它的输出依赖于当前的__输入__和__记忆__
1. $f_{W}表示的实际上是转换关系,指的是从上一个h_{t-1}到h_{t}和x_{t}到h_{t}两条线$
2. $隐藏层的更新：$
$$h_{t} = f_{W}(h_{t-1},x_{t})$$
$$h_{t} = tanh(W_{hh}*h_{t-1}+W_{xh}x_{t})$$
$$y_t=W_{hy}*h_{t}$$

$根据CS231-Lecture14min，we\;use\;the\;same\;function\;f\;and\;same\;parameter\;W$
<img src="./image/rnn.png" style="zoom:50%">

3. 对Hidden layer操作而不对input进行操作的原因在于input叠加只会记住前面一个的内容，而hidden layer可以记得多轮的结果。  

## 2.输入输出理解
1. $参数$
$$RNN(input\_size,hidden\_size,num\_layers,nonlinearity,bias,batch\_first,drop\_out,bi)$$
    + $input\_size：输入数据x特征的的维度。$
    + $hidden\_size:隐藏神经元的数目。$
    + $num\_layers: 循环层的数目，如果这个参数的数目是2的时候，表示将两个RNN叠加在一起，也就是说第二个RNN将第一个RNN的输出作为输入，并计算得到最终的结果。$
    + $nonlinearity：表示采用的非线性操作，tanh或者relu。$
    + <font color='red'>$batch\_first:bool变量，如果被设为True的话，输入的元素的顺序将会被调整为：(batch,seq,feature)，正常的顺序是(seq,batch,feature\_size)$</font>
2. $输入$
    + **input** of the shape `(seq_len, batch, input_size)`: seq_len从NLP的角度进行理解，可以看出是一个句子由多少词语组成；batch表示一次输入多少条句子；input_size表示的是每个词语的词向量的维度。
    + **h_0** of shape `(num_layers * num_directions, batch, hidden_size)`:包含的初始状态下，所有batch中所有隐藏神经元的数目，默认全部被设置为0，如果没有输入的话。
3. $输出$
    + **h_n** `(seq_len, batch, num_directions * hidden_size)`:表示的就是输出，注意这里的输出并不是经过全连接之后的得到的结果，而是hidden_state的值。

4. $理解$
    + RNN的一个输入的size是 `(seq_len, batch, input_size)`，比如例子中的torch.randn(5,1,10),这个5并没有出现在网络的结构中，如何理解这个5。实际上这个5就代表了序列展开后的数目，比如这张图：
    <img src="./image/rnn-1.png" style="zoom:50%">
    展开的t就代表了5，也就是说这个展开的结构实际上是可变的。一个更直观的理解就是输入 '我 是 一个 好人'，这是一句话，这句话中包含了四个词语，['我'，'是'，'一个','好人']，这些词语可以被转化为向量，所以实际上：
    $$
    \begin{align*}
     x_0 & = word2vec(我) \\
     x_1 & = word2vec(是)\\
     x_2 & = word2vec(一个)\\
     x_3 &= word2vec(好人)
     \end{align*}
    $$
<br>视频到 -->39.33min

In [1]:
import torch
import torch.nn as nn

In [2]:
rnn = nn.RNN(input_size=10,hidden_size=20)

In [3]:
input = torch.randn(5,1,10)
output = rnn(input)
print(output[0].size()) # h1 -> 10,1,20

torch.Size([5, 1, 20])


In [15]:
for name,i in rnn.named_parameters():
    print(name,i.size())

weight_ih_l0 torch.Size([20, 10])
weight_hh_l0 torch.Size([20, 20])
bias_ih_l0 torch.Size([20])
bias_hh_l0 torch.Size([20])


# ==========================================================

In [4]:
import torch
from torchtext import data,datasets

In [8]:
TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.float)

In [ ]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [ ]:
train_data

# ==========================================================

In [13]:
import numpy as np
import copy
import matplotlib.pyplot as plt

In [14]:
BIN_DIM = 8
INPUT_DIM = 2 
HIDDEN_DIM = 16
OUTPUT_DIM = 1

alpha = 0.1
iter_num = 10000
log_iter = iter_num // 10
plot_iter = iter_num // 200

In [15]:
largest = pow(2,BIN_DIM)
decimal = np.array([range(largest)],dtype=np.uint8).T
binary = np.unpackbits(decimal,axis=1)

In [16]:
w0 = np.random.normal(0,1,[INPUT_DIM,HIDDEN_DIM])
w1 = np.random.normal(0,1,[HIDDEN_DIM,OUTPUT_DIM])
wh = np.random.normal(0,2,[HIDDEN_DIM,HIDDEN_DIM])

In [17]:
d0 = np.zeros_like(w0)
d1 = np.zeros_like(w1)
dh = np.zeros_like(wh)

In [18]:
errs = list() 
accs = list()
error = 0
accuracy = 0

In [19]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def derivation(output):
    return output * (1 - output)

In [21]:
for i in range(iter_num + 1):
    a_dec = np.random.randint(largest / 2)
    b_dec = np.random.randint(largest / 2)
    c_dec = a_dec + b_dec
    
    a_bin = binary[a_dec]
    b_bin = binary[b_dec]
    c_bin = binary[c_dec]
    
    pred = np.zeros_like(c_bin)
    
    overall_err = 0
    
    output_deltas = list()
    hidden_values = list()
    
    # 初始化隐藏神经元为0
    hidden_values.append(np.zeros(HIDDEN_DIM))
    
    # 用于反向传播时记录未来的梯度更新
    future_delta = np.zeros(HIDDEN_DIM)
    
    # 前向传播
    for pos in range(BIN_DIM)[::-1]:
        X = np.array([[a_bin[pos],b_bin[pos]]])  # shape (1,2)
        Y = np.array([[c_bin[pos]]]) # shape (1,1)
        
        hidden = sigmoid(np.dot(X,w0) + np.dot(hidden_values[-1],wh))
        output = sigmoid(np.dot(hidden,w1))
        
        pred[pos] = np.round(output[0][0])
        
        output_err  = Y - output
        
        output_deltas.append(output_err * derivation(output))
        
        hidden_values.append(hidden)
        
        overall_err += np.abs(output_err[0])
        
    for pos in range(BIN_DIM):
        
        X = np.array([[a_bin[pos],b_bin[pos]]])
        
        hidden = hidden_values[-pos-1]
        pre_hidden = hidden_values[-pos-2]
        
        output_delta = output_deltas[-pos-1]
        # 更新的梯度有两部分组成，一部分是当前网络反向传递过来的梯度，另一部分是由前一网络反向传播过来的梯度
        hidden_delta = (np.dot(future_delta,wh.T)+np.dot(output_delta,w1.T)) * derivation(hidden)
        
        d1+= np.dot(np.atleast_2d(hidden).T,output_delta)
        dh+= np.dot(np.atleast_2d(pre_hidden).T,hidden_delta)
        d0+= np.dot(X.T,hidden_delta)
        
        future_delta = hidden_delta
    
    w1 += alpha * d1
    w0 += alpha * d0
    wh += alpha * dh
    
    
    d1 = 0
    d0 = 0
    dh = 0

    if (i % 100 == 0):
        print('Iter', i)
        print("Error :", overall_err)
        print("Pred :", pred)
        print("True :", c_bin)
        print('----------')

Iter 0
Error : [4.77943112]
Pred : [0 0 0 0 0 0 0 1]
True : [0 1 1 1 0 1 1 1]
----------
Iter 100
Error : [4.05596263]
Pred : [0 0 0 0 1 0 0 1]
True : [1 1 0 1 0 0 1 0]
----------
Iter 200
Error : [4.07075387]
Pred : [0 0 1 1 1 1 1 0]
True : [0 1 0 1 0 0 1 0]
----------
Iter 300
Error : [4.09371011]
Pred : [1 1 0 1 1 1 1 1]
True : [1 0 1 0 1 0 0 1]
----------
Iter 400
Error : [3.92993139]
Pred : [1 0 1 1 1 1 1 1]
True : [0 1 1 0 1 0 0 1]
----------
Iter 500
Error : [3.81050789]
Pred : [1 1 1 1 1 0 1 1]
True : [1 0 1 1 0 0 0 1]
----------
Iter 600
Error : [3.64338068]
Pred : [1 1 1 1 1 1 1 1]
True : [1 1 0 1 0 1 0 1]
----------
Iter 700
Error : [3.5614443]
Pred : [1 0 1 1 1 1 0 1]
True : [0 1 1 1 1 1 0 0]
----------
Iter 800
Error : [4.00963678]
Pred : [1 1 1 1 1 1 1 1]
True : [1 1 1 0 0 0 1 1]
----------
Iter 900
Error : [3.76532686]
Pred : [1 1 1 1 1 1 1 1]
True : [0 1 1 1 1 1 0 0]
----------
Iter 1000
Error : [3.58114029]
Pred : [0 1 0 0 1 0 0 0]
True : [0 1 0 1 1 0 1 0]
----------
I

Iter 9000
Error : [0.42024033]
Pred : [1 1 1 1 0 0 1 1]
True : [1 1 1 1 0 0 1 1]
----------
Iter 9100
Error : [0.49175699]
Pred : [0 1 0 1 1 0 0 0]
True : [0 1 0 1 1 0 0 0]
----------
Iter 9200
Error : [0.40858913]
Pred : [1 0 1 0 0 0 1 0]
True : [1 0 1 0 0 0 1 0]
----------
Iter 9300
Error : [0.38811999]
Pred : [1 0 0 0 1 1 0 1]
True : [1 0 0 0 1 1 0 1]
----------
Iter 9400
Error : [0.3751637]
Pred : [0 1 1 1 0 0 0 0]
True : [0 1 1 1 0 0 0 0]
----------
Iter 9500
Error : [0.46203923]
Pred : [0 1 0 0 0 1 0 1]
True : [0 1 0 0 0 1 0 1]
----------
Iter 9600
Error : [0.492297]
Pred : [0 1 0 1 0 1 1 0]
True : [0 1 0 1 0 1 1 0]
----------
Iter 9700
Error : [0.48300201]
Pred : [1 1 0 1 1 1 0 0]
True : [1 1 0 1 1 1 0 0]
----------
Iter 9800
Error : [0.54560361]
Pred : [1 0 1 0 0 1 1 0]
True : [1 0 1 0 0 1 1 0]
----------
Iter 9900
Error : [0.52384517]
Pred : [1 0 0 0 1 0 1 0]
True : [1 0 0 0 1 0 1 0]
----------
Iter 10000
Error : [0.3875008]
Pred : [0 1 1 0 1 1 1 1]
True : [0 1 1 0 1 1 1 1]
--